**initialize**(False,'DIRECT','3-21G','1','H2O1.xyz',
                   func_high,func_low,*eri*='nofit')
                   
//...

**return** basis_gen,basis_acc, molstring,psi4mol,wfn,jkbase

input: (in order of appearence)
- jkflag : if True; native Psi4 JK class is used to compute J and K matrices
- scf_type : set the scf type calculation ('DIRECT', 'DF_MEM', 'DF_DISK', 'PK'). It affects the type of
  integrals being used (if jkflag=True) and the initial density guess
- obs1 : orbital basis set string "genbasis;El1:basis1;El2:basis2 .." 
- frag_spec : [type: str] specify a semicolon-seperated id (integer) sequence of fragments to include in the high
   level region
- fgeom : geometry file
- func1 : functional for 'high-level-theory' frag
- func2 : functional for 'low-level-theory' frag
- charge : charge of the whole system. (Singlet state is assumed)
- eri : when specified, set the the type of electron-repulsion integrals in use ('nofit'; 'fit')

output:
- basis_acc : the basis object of the 'high-level-theory' frag
- basis_gen : the basis object of the 'low-level-theory' frag
- molstring : a string containing geometry of the entire system
- psi4_mol  : psi4 geometry object of  the entire system
- wfn: psi4 wavefunction object
- jkbase : the jkfactory object

**TASKS**:
- from basis_gen get the Psi4 mints object (mints is responsible for integrals in Psi4NumPy, see manual)
- get overlap matrix (S) in the AO basis (basis_gen)
- set the U matrix -> block orthogonalize the overlap

The density matrix in the AO basis and the corresponding matrix in the BO basis are related by
$$ D^{ao} = U \tilde{D} U^{\dagger} $$
Morover $ U^{-1} D^{ao} (U^{\dagger})^{-1} = \tilde{D} $. $U$ is not orthogonal ($U^{-1} != U^{\dagger}$).

Correspondingly the matrix representation of a given operator in the BO basis can be obtained from the AO basis representation as:
$$ \tilde{O} = U^{\dagger} O^{ao} U $$

In [2]:
import os
import sys
import psi4
import numpy as np

sys.path.insert(0, "./common")
modpaths = os.environ.get('MODS_PATH')

if modpaths is not None :
    for path in modpaths.split(";"):
        sys.path.append(path)
from scf_run import run
from init_run import initialize
from Fock_helper import fock_factory
print("test mixed basis / functionals")
func_high = 'b3lyp'
func_low = 'b3lyp'
bset,bsetH, molelecule_str, psi4mol, wfn, jkobj = initialize(False,'DIRECT','3-21G','1','H2O1.xyz',\
                   func_high,func_low,eri='nofit')

mints = psi4.core.MintsHelper(bset)
#I = np.array(mints.ao_eri())

H = np.array(mints.ao_kinetic())+ np.array(mints.ao_potential())
S = np.array(mints.ao_overlap())
numbas = bset.nbf() #get the number of basis function belonging to bset (the low-level-theory basis)

nbfA = bsetH.nbf() #get the number of basis function belonging to bsetH (the high-level-theory basis)

#make U matrix for blend basis(cc-pvdz+3-21G)
U = np.eye(numbas)
S11=S[:nbfA,:nbfA]
S11_inv=np.linalg.inv(S11)
S12 =S[:nbfA,nbfA:]
P=np.matmul(S11_inv,S12)
U[:nbfA,nbfA:]=-1.0*P

#S block orthogonal
Stilde= np.matmul(U.T,np.matmul(S,U))


mtest=np.zeros((nbfA,(numbas-nbfA)))
print("AB block of overlap (in BO basis) is zero: %s" %(np.allclose(Stilde[:nbfA,nbfA:],mtest)))


test mixed basis / functionals
High Level functional : b3lyp

Low Level functional : b3lyp

number of fragmemts: 2

functions in general basis: 13
functions in subsys1: 11
The low-level-theory functional requires HF exch

Size of the ERI tensor will be 0.00 GB.
eri n. axis: 4
eri is instance: <class 'numpy.ndarray'>

jkfactory -> eri tensor provided, real-time HF exch required: False
AB block of overlap (in BO basis) is zero: True


---------------------------------------------------------
**Use fock_factory to get the Fock in BO basis**

A fock_factory instance can be initialized as:

fock_engine = **fock_factory**(jk_fact,Hmat,ovapm,*funcname*=None,*basisobj*=None,*exmodel*=0)

input: 

- jk_fact: the jk object obtained from the "initialize" function
- Hmat: the $ H_{core}$ matrix in the AO basis
- ovapm: the basis function overlap matrix. Only needed when we work in the density matrix framework, i.e
  forming $ SDS $, and diagonalizing the "density operator" we get natural orbitals. ovapm can be either $\tilde{S}$   or $S^{ao}$ depending on the basis we are working, respectively, BO or AO basis
- funcname: the functional corresponding to the low level theory, when BO embedding is intended. Otherwise is the     usual functional in a supermolecular calculation.
- basis object: same as funcname
- exmodel : exchange model, 0 is assumed, see Manby-Miller and Parkhill papers

fock_factory has many methods:

- **get_bblock_Fock**(self,*Cocc*=None,*Dmat*=None,*func_acc*=None,*basis_acc*=None,*U*=None,*return_ene*=False). This function can accept either the matrix containing MO coefficients or the density matrix (in the appropriate basis). **basis_acc** and **func_acc** denote the 'high-level-theory' basis object and functional respectively. $U$ has to be passed as numpy.ndarray 
- **get_Fock**(self,*Cocc*=None,*Dmat*=None,*return_ene*=False). This method can be used to get the usual supermolecular Fock matrix corresponding to $C_{occ}$/$D_{mat}$, *funcname* and *basisobj* (see the intialization)


For this "b3lyp-in-b3lyp" test we can retrive the orbitals ($C_{occ}$) from the Psi4 wfn object.

**TASK**:
- transform the orbitals from the AO basis to the BO basis (using $U^{-1}$)
-  define an input density matrix (BO basis)
- define the reference Fock (from Psi4 wfn), and do the trasformation $\tilde{F} = U^{\dagger} F^{ao} U$ 
- compare the two matrices

In [3]:
Cocc = np.array(wfn.Ca_subset('AO','OCC'))

try:
    U_inv = np.linalg.inv(U)
except np.linalg.LinAlgError:
    print("Error in numpy.linalg.inv of inputted matrix")

Cocc = np.matmul(U_inv,Cocc)
Dinput = np.matmul(Cocc,Cocc.T)
fockbase = fock_factory(jkobj,H,Stilde,funcname=func_low,basisobj=bset) #Stilde as overlap matrix
F_bblock = fockbase.get_bblock_Fock(Dmat=Dinput,func_acc=func_high,basis_acc=bsetH,U=U)

print("F(BO) dim: %i,%i\n" % (F_bblock.shape[0],F_bblock.shape[1]))
Test_H = np.matmul(U.T,np.matmul(wfn.Fa(),U))

test = np.allclose(F_bblock,Test_H,atol=1.0e-12)
print("test GGA/hybrid Fock Block-Orth.: Passed .... %s\n" % test)


F(BO) dim: 13,13

test GGA/hybrid Fock Block-Orth.: Passed .... True

